In [6]:
import json
import boto3
import random
import bisect
import pandas as pd

# DynamoDBオブジェクト
dynamodb = boto3.resource('dynamodb')

# 楽曲をリコメンドする関数
def music_recommend(ans,df):# 質問の回答/データセットを引数とする
    data=['name', 'artist', 'release_date', 'length', 'popularity', 'danceability', 'acousticness', 'energy', 'loudness', 'tempo',  'valence', 'url', 'image', 'audio', 'iine']
    df1 = df[data]
    min_point=10000
    df_sort_lst=[]
    for i in range(6):
        df_sort_lst.append(df1.sort_values(by=data[i+5], ascending=True))
        
    for i in range(len(df1)):
        point=0
        cnt=len(df1)/5
        for j in range(len(ans)-1):
            if ans[j+1]==5:
                point+=random.uniform(0, 1)
            else:
                df_sort=df_sort_lst[j]
                rand=random.uniform(df_sort.values[int(ans[j+1]*cnt)][j+5], df_sort.values[int((ans[j+1]+1)*cnt)-1][j+5])
                point+=pow(df1.values[i][j+5]-rand,2)
        if point<min_point:
            min_point=point
            min_i=i
        
    min_values=[]
    for k in range(len(ans)-1):
        min_values.append(df1.values[min_i][k+5])
    rank=[]
    for i in range(6):
        df_sort=df_sort_lst[i]
        cnt=bisect.bisect(list(df_sort[data[i+5]].values),min_values[i])
        rank.append(cnt)

    music_data=[]
    music_data.append(df1.values[min_i][0]) # リコメンドする楽曲
    music_data.append(df1.values[min_i][1]) # リコメンドする楽曲のアーティスト名
    music_data.append(df1.values[min_i][11]) # リコメンドする楽曲のurl
    music_data.append(df1.values[min_i][12]) # リコメンドする楽曲のimage
    music_data.append(df1.values[min_i][13]) # リコメンドする楽曲のaudio
    music_data.append(min_values) # リコメンドする楽曲の数値(リスト)
    music_data.append(rank) # リコメンドする楽曲の数値の順位(リスト)(その音楽データの中での順位)
    music_data.append(df1.values[min_i][14]) # リコメンドする楽曲のいいね数
    return music_data # 楽曲データを返す

# メインの処理
event={"genre":"1","q1":"1","q2":"5","q3":"3","q4":"3","q5":"1","q6":"0"}
# 質問の解答を得る
ans = [int(event["genre"]),int(event["q1"]),int(event["q2"]),int(event["q3"]),int(event["q4"]),int(event["q5"]),int(event["q6"])]
df = pd.read_csv('music_data/spotify_data_JP_famous_mm_cluster.csv')

# 楽曲のレコメンド
music=music_recommend(ans,df)
Q=[['好き','やや好き','普通','やや嫌い','嫌い','特に好みはない'],['強め','やや強め','普通','やや弱め','弱め','特に好みはない']
,['強め','やや強め','普通','やや弱め','弱め','特に好みはない'],['強め','やや強め','普通','やや弱め','弱め','特に好みはない']
,['速め','やや速め','普通','やや遅め','遅め','特に好みはない'],['楽観的','やや楽観的','普通','やや悲観的','悲観的','どちらでも良い']]
Answer=[]
for i in range(6):
    Answer.append(Q[i][ans[i+1]])

value = {"v1":music[5][0],"v2":music[5][1],"v3":music[5][2],"v4":music[5][3],"v5":music[5][4],"v6":music[5][5]}
rank = {"r1":music[6][0],"r2":music[6][1],"r3":music[6][2],"r4":music[6][3],"r5":music[6][4],"r6":music[6][5]}
answer = {"q1":Answer[0],"q2":Answer[1],"q3":Answer[2],"q4":Answer[3],"q5":Answer[4],"q6":Answer[5]}
music_data={"name":music[0],"artist":music[1],"url":music[2],"image":music[3],"audio":music[4],"value":value,"rank":rank,"answer":answer,"size":len(df),"iine":music[7]}

music_data

{'name': '悪魔の子',
 'artist': 'ヒグチアイ',
 'url': 'https://open.spotify.com/track/2TVFbLCdzrX0RdhWNWJmdp',
 'image': 'https://i.scdn.co/image/ab67616d000048517e2a5ab7f69511ef36550b16',
 'audio': 'https://p.scdn.co/mp3-preview/799901c1ece5d1e4b820d2563e4d6675f48b26a3?cid=18308f0d5b8049c89ee330d9cfa4bc58',
 'value': {'v1': 0.473474801,
  'v2': 0.298240909,
  'v3': 0.649827784,
  'v4': 0.620519498,
  'v5': 0.147390473,
  'v6': 0.191144708},
 'rank': {'r1': 175, 'r2': 251, 'r3': 154, 'r4': 109, 'r5': 70, 'r6': 21},
 'answer': {'q1': 'やや好き',
  'q2': '特に好みはない',
  'q3': 'やや弱め',
  'q4': 'やや弱め',
  'q5': 'やや速め',
  'q6': '楽観的'},
 'size': 391,
 'iine': 0}

In [41]:
import json
df_Global = pd.read_csv('music_data/spotify_data_JP_famous_mm_cluster.csv')
df_Japan = pd.read_csv('music_data/spotify_data_global_mm_cluster.csv')
df_Global["iine"][0]=1
df_Global["iine"][390]=2
df_Global["iine"][96]=3
df_Japan["iine"][0]=1
df_Japan["iine"][88]=2
df_Japan["iine"][45]=3
df_Global=df_Global.sort_values(by='iine', ascending=False)
df_Japan=df_Japan.sort_values(by='iine', ascending=False)
data=['name', 'artist','url', 'image', 'audio', 'iine']
music_Global=[]
music_Japan=[]
for i in range(3):
    music_Global.append(df_Global[data].values[i])
    music_Japan.append(df_Japan[data].values[i])
rank1={"global":music_Global[0],"japan":music_Japan[0]}
rank2={"global":music_Global[1],"japan":music_Japan[1]}
rank3={"global":music_Global[2],"japan":music_Japan[2]}
music_data={"rank1":rank1,"rank2":rank2,"rank3":rank3}
music_data['rank1']['global']

C:\Users\ktaak\AppData\Local\Temp\ipykernel_17124\4026763925.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Global["iine"][0]='1'
C:\Users\ktaak\AppData\Local\Temp\ipykernel_17124\4026763925.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Japan["iine"][0]='1'


TypeError: '<' not supported between instances of 'int' and 'str'

In [50]:
import json
json_Global = [{
      "artist": "ワン・ダイレクション",
      "valence": "0.916985254",
      "danceability": "0.678977273",
      "iine": "2",
      "acousticness": "0.008624125",
      "url": "https://open.spotify.com/track/1T07kafGtJbDvosN0nu4Q2",
      "name": "What Makes You Beautiful",
      "popularity": "0",
      "release_date": "2011/10/28",
      "cluster_pred": "6",
      "length": "0.324014791",
      "energy": "0.77764566",
      "image": "https://i.scdn.co/image/ab67616d0000485136d38e3cf2ebac3fb7094b05",
      "audio": "なし",
      "loudness": "0.907005355",
      "id": "2",
      "tempo": "0.516073905"
    },
    {
      "artist": "ケイティ・ペリー",
      "valence": "0.378481704",
      "danceability": "0.768465909",
      "iine": "8",
      "acousticness": "0.003935893",
      "url": "https://open.spotify.com/track/6bUKd5y4eThkXvfJFSAUJ1",
      "name": "California Gurls",
      "popularity": "0.43",
      "release_date": "2010/1/1",
      "cluster_pred": "5",
      "length": "0.43679196",
      "energy": "0.747919144",
      "image": "https://i.scdn.co/image/ab67616d0000485127271ebe78e28a2ff83eaa47",
      "audio": "なし",
      "loudness": "0.819438414",
      "id": "1",
      "tempo": "0.516115184"
    },
    {
      "artist": "ブルーノ・マーズ",
      "valence": "0.981430912",
      "danceability": "0.776988636",
      "iine": "5",
      "acousticness": "0.317401881",
      "url": "https://open.spotify.com/track/386RUes7n1uM1yfzgeUuwp",
      "name": "The Lazy Song",
      "popularity": "0.77",
      "release_date": "2010/10/5",
      "cluster_pred": "6",
      "length": "0.291861842",
      "energy": "0.697978597",
      "image": "https://i.scdn.co/image/ab67616d00004851f60070dce96a2c1b70cf6ff0",
      "audio": "https://p.scdn.co/mp3-preview/7d6bedf685cae3d96d866550d0d2c412532f4f89?cid=18308f0d5b8049c89ee330d9cfa4bc58",
      "loudness": "0.71710812",
      "id": "30",
      "tempo": "0.928158642"
    }
    ]
# df_Japan = pd.read_csv('music_data/spotify_data_global_mm_cluster.csv')
json_Global = sorted(json_Global, key=lambda k: k['iine'], reverse=True)
json_Global[0]

{'artist': 'ケイティ・ペリー',
 'valence': '0.378481704',
 'danceability': '0.768465909',
 'iine': '8',
 'acousticness': '0.003935893',
 'url': 'https://open.spotify.com/track/6bUKd5y4eThkXvfJFSAUJ1',
 'name': 'California Gurls',
 'popularity': '0.43',
 'release_date': '2010/1/1',
 'cluster_pred': '5',
 'length': '0.43679196',
 'energy': '0.747919144',
 'image': 'https://i.scdn.co/image/ab67616d0000485127271ebe78e28a2ff83eaa47',
 'audio': 'なし',
 'loudness': '0.819438414',
 'id': '1',
 'tempo': '0.516115184'}

In [35]:
Json={
  "name": "taro",
  "age": 23,
  "sex": "man",
  "hobby": [
    "running",
    "swimming",
    "singing"
  ]
}
Json

{'name': 'taro',
 'age': 23,
 'sex': 'man',
 'hobby': ['running', 'swimming', 'singing']}